## Test Data

Kbank Load Products ข้อมูลจาก web site

Product List
- สินเชื่อส่วนบุคคล K Pay Later
- สินเชื่อเงินด่วนเพื่อธุรกิจ Xpress Loan (XPL) แบบผ่อนระยะยาวเพื่อธุรกิจ (Loan)
- สินเชื่อเงินด่วนเพื่อธุรกิจ Xpress Loan (XPL) แบบใช้หมุนเวียนเพื่อธุรกิจ (OD)

In [ ]:
# Install Vertex AI LLM SDK, langchain and dependencies
! pip install google-cloud-aiplatform langchain==0.0.323 chromadb==0.3.26 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0 pandas datasets google-api-python-client pypdf faiss-cpu transformers config --upgrade --user

In [1]:
# Utils
from langchain.schema import HumanMessage, SystemMessage
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.chat_models import ChatVertexAI
from google.cloud import aiplatform
import time
from typing import List

# LangChain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")

# Vertex AI

print(f"Vertex AI SDK version: {aiplatform.__version__}")

LangChain version: 0.0.323
Vertex AI SDK version: 1.35.0


In [2]:
# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

In [4]:
# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Chat
chat = ChatVertexAI()

# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

Simple test

In [5]:
llm("Hi, what is your name?")

'I am powered by PaLM 2, which stands for Pathways Language Model 2, a large language model from Google AI.'

## Example 1: Load CSV Data and search 1 item for the most matched with user query

In [6]:
from langchain.document_loaders.csv_loader import CSVLoader

In [31]:
data = []

In [34]:
loader = CSVLoader(file_path="./data/kbank-loan-products.csv")

data = loader.load()

In [35]:
len(data)

3

In [36]:
print(data)

[Document(page_content='Product Name: สินเชื่อเงินด่วน\nคำอธิบาย: กู้เงินด่วน เงินก้อนทันใจ กู้ง่าย อนุมัติไว ได้เงินทันที\nจุดเด่น: รู้ผลอนุมัติไวสุดใน 15 นาที - 1 วันทำการ อนุมัติปุ๊บโอนเงินเข้าบัญชีให้ทันที เมื่อสมัครผ่าน G-BANK App อาชีพอิสระ ไม่ได้เป็นพนักงานเงินเดือนก็สมัครได้\nวงเงินสูงสุด: สูงสุด 5 เท่าของรายได้ เช่น วงเงิน 37,500 บาท สำหรับพนักงานรายได้ 7,500 บาทต่อเดือน\nระยะเวลาอนุมัติ: 15 นาที - 1 วันทำการ\nได้รับเงินภายใน: เงินเข้าบัญชีให้ทันที เมื่อสมัครผ่าน G-BANK App ตั้งแต่เวลา 09.00 – 19.30 น. ของทุกวัน\nอัตราดอกเบี้ยต่อปี: 17% - 25%\nค่าธรรมเนียม: ไม่มี\nรูปแบบการชำระ การผ่อน: ผ่อนชำระรายเดือน อัตราดอกเบี้ยลดต้น ลดดอก\nช่องทางการสมัคร: G-BANK App หรือ สาขา\nเอกสารแสดงตัว: สมัครผ่าน G-BANK App ไม่ต้องใช้เอกสาร หรือ\nสำเนาบัตรประชาชน หรือ\nสำเนาบัตรประจำตัวข้าราชการ หรือ รัฐวิสาหกิจ (ต้องแนบสำเนาทะเบียนบ้านประกอบด้วย)\nเอกสารแสดงรายได้: สลิปเงินเดือน', metadata={'source': './data/kbank-loan-products.csv', 'row': 0}), Document(page_content='Product Name: บัตรเงินด่วน\nค

In [37]:
# Store docs in local vectorstore as index
# it may take a while since API is rate limited
from langchain.vectorstores import Chroma

db = Chroma.from_documents(data, embeddings)

Waiting


In [20]:
# Expose index to the retriever
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [14]:
# Create chain to answer questions
from langchain.chains import RetrievalQA

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

In [23]:
query = "เป็นพนักงานประจำแนะนำสินเชื่อตัวไหน?"
result = qa({"query": query})
print(result)

{'query': 'เป็นพนักงานประจำแนะนำสินเชื่อตัวไหน?', 'result': 'สินเชื่อเงินด่วนเพื่อธุรกิจ แบบผ่อนระยะยาวเพื่อธุรกิจ', 'source_documents': [Document(page_content='Product Name: สินเชื่อเงินด่วนเพื่อธุรกิจ แบบผ่อนระยะยาวเพื่อธุรกิจ (Loan)\nคำอธิบาย: สินเชื่อเงินด่วนเพื่อธุรกิจ แบบผ่อนระยะยาวเพื่อธุรกิจ\nจุดเด่น: เงินก่อนโอนเข้าบัญชี ดอกเบี้ยลดต้นลดดอก\nปิดวงเงินก่อนได้ ไม่มีค่าธรรมเนียม\nวงเงินสูงสุด: 20,000 - 1,000,000 บาท\nระยะเวลาอนุมัติ: 5 นาที\nได้รับเงินภายใน: เงินโอนเข้าบัญชีออมทรัพย์ที่เลือกไว้ทันทีหลังอนุมัติ\nอัตราดอกเบี้ยต่อปี: คงที่สูงสุดไม่เกิน 25% ต่อปี\nค่าธรรมเนียม: มีค่าธรรมเนียมรายปี\nมีค่าอากรแสดมป์\nค่าธรรมเนียมการจัดการให้กู้\nรูปแบบการชำระ การผ่อน: ผ่อนเท่ากันทุกเดือน\nระยะเวลาผ่อนชำระสูงสุด 5 ปี\nช่องทางการสมัคร: G-BANK App หรือ สาขา\nเอกสารแสดงตัว: บัตรประจำตัวประชาชน บัญชีเงินฝากธนาคาร\nเอกสารแสดงรายได้: กรณี มีบัญชีกับธนาคาร และเดินบัญชีขั้นต่ำ 6 เดือนขึ้นไป ไม่จำเป็นต้องยื่นเอกสารเพิ่มเติม\nกรณี เป็นลูกค้าใหม่ (มีบัญชีกับธนาคารน้อยกว่า 6 เดือน) หรือ อยากแสดงรายไ

In [24]:
from langchain.vectorstores import FAISS

# Embed your texts
db = FAISS.from_documents(data, embeddings)

# Init your retriever. Asking for just 1 document back
retriever = db.as_retriever()

Waiting


In [25]:
docs = retriever.get_relevant_documents(
    "เป็นพนักงานประจำ อยากได้เงินสดด่วนครับ"
)

In [26]:
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))

Product Name: สินเชื่อเงินด่วนเพื่อธุรกิจ แบบผ่อนระยะยาวเพื่อธุรกิจ (Loan)
คำอธิบาย: สินเชื่อเงินด่วนเพื่อธุรกิจ แบบผ่อนระยะยาวเพื่อธุรกิจ
จุดเด่น: เงินก่อนโอนเข้าบัญชี ดอกเบี้ยลดต้นลดดอก
ปิดวงเงินก่อ

Product Name: สินเชื่อเงินด่วนเพื่อธุรกิจ แบบใช้หมุนเวียนเพื่อธุรกิจ (OD)
คำอธิบาย: สินเชื่อเงินด่วนเพื่อธุรกิจ แบบใช้หมุนเวียนเพื่อธุรกิจ
จุดเด่น: เสียดอกเบี้ยเฉพาะวงเงินที่เบิกใช้
วงเงินสูงสุด: 20,0


In [27]:
print("\n\n".join([x.page_content[:200] for x in docs]))

Product Name: สินเชื่อเงินด่วนเพื่อธุรกิจ แบบผ่อนระยะยาวเพื่อธุรกิจ (Loan)
คำอธิบาย: สินเชื่อเงินด่วนเพื่อธุรกิจ แบบผ่อนระยะยาวเพื่อธุรกิจ
จุดเด่น: เงินก่อนโอนเข้าบัญชี ดอกเบี้ยลดต้นลดดอก
ปิดวงเงินก่อ

Product Name: สินเชื่อเงินด่วนเพื่อธุรกิจ แบบใช้หมุนเวียนเพื่อธุรกิจ (OD)
คำอธิบาย: สินเชื่อเงินด่วนเพื่อธุรกิจ แบบใช้หมุนเวียนเพื่อธุรกิจ
จุดเด่น: เสียดอกเบี้ยเฉพาะวงเงินที่เบิกใช้
วงเงินสูงสุด: 20,0

Product Name: บัตรเงินด่วน
คำอธิบาย: สินเชื่อบัตรเงินด่วน วงเงินอนุมัติสูง สินเชื่อเงินสด บัตรกดเงินสด บัตรผ่อนสินค้า รูดแทนเงินสด
จุดเด่น: บัตรกดเงินสุด สมัครง่าย ใช้บัตรประชาชนใบเดียว กู้ง่าย ไม่ต้อ

Product Name: สินเชื่อเงินด่วน
คำอธิบาย: กู้เงินด่วน เงินก้อนทันใจ กู้ง่าย อนุมัติไว ได้เงินทันที
จุดเด่น: รู้ผลอนุมัติไวสุดใน 15 นาที - 1 วันทำการ อนุมัติปุ๊บโอนเงินเข้าบัญชีให้ทันที เมื่อสมัครผ่าน G
